In [21]:
# Import required libraries
import pandas as pd  # For handling CSV files and dataframes
from datetime import datetime  # For working with date and time
import uuid  # For generating unique booking IDs

# Define a class to handle Mass Booking operations using Object-Oriented Programming (OOP)
class MassBooking:
    def __init__(self, name=None, mass_intention=None, type_of_mass=None, start_date=None, end_date=None):
        """
        Initialize a new booking instance with optional input values.
        Sets up default CSV path and column headers.
        Loads existing bookings from file (if available).
        """
        self.name = name
        self.mass_intention = mass_intention
        self.type_of_mass = type_of_mass
        self.start_date = start_date
        self.end_date = end_date
        self.date_booked = datetime.now()  # Capture the date and time of booking
        self.file_path = 'mass_booked.csv'  # CSV file to store bookings
        self.columns = ['Booking_ID', 'Name', 'Mass Intention', 'Mass_Type', 'Start Date', 'End Date', 'Date_Booked']
        self.load_csv()  # Load or initialize CSV file

    def load_csv(self):
        """
        Load the existing bookings from CSV file if it exists,
        otherwise create an empty DataFrame with defined columns.
        """
        try:
            self.df = pd.read_csv(
                self.file_path,
                parse_dates=['Start Date', 'End Date', 'Date_Booked']
            )
        except FileNotFoundError:
            self.df = pd.DataFrame(columns=self.columns)

    def save_csv(self):
        """
        Save the current state of the DataFrame to the CSV file.
        """
        self.df.to_csv(self.file_path, index=False)

    def introductory_page(self):
        """
        Display welcome message and prompt the user for their action choice.
        """
        print('\nWelcome to St. Cyprian Mass Booking Management System')
        print('\nPlease note each entry is peculiar to a particular mass')
        self.user_introduction = input(
            'What would you like to do?\n'
            'Add a new mass intention (keyword: add)\n'
            'Update a mass intention (keyword: upd)\n'
            'Delete a mass intention (keyword: del)\n'
            'Check booked intentions (keyword: check)\n'
            'Exit the program (keyword: exit)\n> '
        ).lower()

    def validate_dates(self):
        """
        Ensure start and end dates are valid and in the correct format.
        If not, prompt the user to re-enter them.
        """
        while True:
            try:
                if isinstance(self.start_date, str):
                    self.start_date = datetime.strptime(self.start_date, "%Y/%m/%d")
                if isinstance(self.end_date, str):
                    self.end_date = datetime.strptime(self.end_date, "%Y/%m/%d")
                if self.start_date > self.end_date:
                    raise ValueError("Start date must be before or equal to end date.")
                return True
            except ValueError as e:
                print(f'Invalid date: {e}. Please use YYYY/MM/DD and ensure start date <= end date.')
                self.start_date = input("Enter the start date (YYYY/MM/DD): ").strip()
                self.end_date = input("Enter the end date (YYYY/MM/DD): ").strip()

    def validate_type_mass(self):
        """
        Validate the type of mass entered by the user against a predefined list.
        Prompt again if invalid.
        """
        valid_masses = ['Morning Mass', 'Afternoon Mass', 'Evening Mass', 'First Mass', 'Second Mass', 'Special Mass']
        while True:
            if isinstance(self.type_of_mass, str) and self.type_of_mass.title() in valid_masses:
                self.type_of_mass = self.type_of_mass.title()
                break
            else:
                print(f'Invalid type of mass: {self.type_of_mass}. Valid options are: {valid_masses}')
                self.type_of_mass = input("Enter the type of mass (e.g., Morning Mass): ").strip()
        return True

    def generate_booking_id(self):
        """
        Generate a unique 8-character booking ID using UUID.
        """
        return uuid.uuid4().hex[:8]

    def adding_intention(self):
        """
        Add a new mass intention by prompting the user for all required details.
        Validates dates and type, appends new record to DataFrame, and saves it.
        """
        # Prompting user inputs
        self.name = input("Enter your name: ").strip()
        self.mass_intention = input("Enter the mass intention: ").strip()
        self.type_of_mass = input("Enter the type of mass (e.g., Morning Mass): ").strip()
        self.start_date = input("Enter the start date (YYYY/MM/DD): ").strip()
        self.end_date = input("Enter the end date (YYYY/MM/DD): ").strip()

        # Validation
        self.validate_dates()
        self.validate_type_mass()

        # Booking ID and new entry creation
        booking_id = self.generate_booking_id()
        new_entry = pd.DataFrame([[
            booking_id,
            self.name,
            self.mass_intention,
            self.type_of_mass,
            self.start_date,
            self.end_date,
            self.date_booked
        ]], columns=self.columns)

        # Add new booking to DataFrame and save
        self.df = pd.concat([self.df, new_entry], ignore_index=True)
        self.save_csv()
        print(f"Booking added successfully. Booking ID: {booking_id}")

    def update_intention(self):
        """
        Update an existing booking based on the date and name filters.
        Prompts user to edit any field for a selected booking ID.
        """
        # Prompt for filtering by booking date
        while True:
            date_filter_str = input("Enter the date the mass was booked (YYYY/MM/DD): ")
            try:
                date_filter = datetime.strptime(date_filter_str, "%Y/%m/%d")
                break
            except ValueError:
                print("Invalid date format. Please use YYYY/MM/DD.")

        # Optional name filter
        name_filter = input("Enter the name used for the booking (optional, press Enter to skip): ").title()
        filtered = self.df[self.df['Date_Booked'].dt.date == date_filter.date()]
        if name_filter:
            filtered = filtered[filtered['Name'] == name_filter]

        # Check if filtered result is empty
        if filtered.empty:
            print("No matching records found.")
            return

        # Display matching records
        print("\nMatching Records:\n", filtered[['Booking_ID', 'Name', 'Mass Intention', 'Start Date', 'End Date']])
        booking_id = input("Enter the Booking ID of the intention to update: ")

        # Validate booking ID
        if booking_id not in self.df['Booking_ID'].values:
            print("Invalid Booking ID.")
            return

        # Get the row index to update
        idx = self.df.index[self.df['Booking_ID'] == booking_id][0]
        print("Current Record:\n", self.df.loc[idx])

        # Loop through each column for possible update
        for col in ['Name', 'Mass Intention', 'Mass_Type', 'Start Date', 'End Date']:
            new_value = input(f"Update {col}? (press Enter to skip): ")
            if new_value:
                if col in ['Start Date', 'End Date']:
                    try:
                        self.df.at[idx, col] = datetime.strptime(new_value, "%Y/%m/%d")
                    except ValueError:
                        print(f"Invalid date for {col}. Skipping update.")
                else:
                    # Apply title casing where needed
                    self.df.at[idx, col] = new_value.title() if col in ['Name', 'Mass_Type'] else new_value

        self.save_csv()
        print("Booking updated successfully.")

    def delete_intention(self):
        """
        Delete an existing booking by filtering with date and (optional) name.
        User confirms deletion by entering the booking ID.
        """
        while True:
            date_filter_str = input("Enter the date the mass was booked (YYYY/MM/DD): ")
            try:
                date_filter = datetime.strptime(date_filter_str, "%Y/%m/%d")
                break
            except ValueError:
                print("Invalid date format. Please use YYYY/MM/DD.")

        name_filter = input("Enter the name used for the booking (optional, press Enter to skip): ").title()
        filtered = self.df[self.df['Date_Booked'].dt.date == date_filter.date()]
        if name_filter:
            filtered = filtered[filtered['Name'] == name_filter]

        if filtered.empty:
            print("No matching records found.")
            return

        print("\nMatching Records:\n", filtered[['Booking_ID', 'Name', 'Mass Intention', 'Start Date', 'End Date']])
        booking_id = input("Enter the Booking ID of the intention to delete: ")

        if booking_id not in self.df['Booking_ID'].values:
            print("Invalid Booking ID.")
            return

        # Remove the record and save
        self.df = self.df[self.df['Booking_ID'] != booking_id]
        self.save_csv()
        print("Booking deleted successfully.")

    def check_bookings(self):
        """
        Check all bookings that fall within a particular date.
        Shows details of all intentions valid on that date.
        """
        while True:
            check_date_str = input("Enter the date to check for mass intentions (YYYY/MM/DD): ")
            try:
                check_date = datetime.strptime(check_date_str, "%Y/%m/%d")
                break
            except ValueError:
                print("Invalid date format. Please use YYYY/MM/DD.")

        # Ensure date columns are in datetime format
        self.df['Start Date'] = pd.to_datetime(self.df['Start Date'], errors='coerce')
        self.df['End Date'] = pd.to_datetime(self.df['End Date'], errors='coerce')

        # Filter for bookings that cover the given date
        matching = self.df[
            (self.df['Start Date'] <= check_date) &
            (self.df['End Date'] >= check_date)
        ]

        if matching.empty:
            print("No mass intentions booked for that date.")
        else:
            print(f"\nMass intentions for {check_date_str}:")
            print(matching[['Booking_ID', 'Name', 'Mass Intention', 'Mass_Type', 'Start Date', 'End Date']])


In [ ]:
# Entry point of the program
# This conditional ensures that this block only runs when the script is executed directly,
# and not when it's imported as a module elsewhere.
if __name__ == "__main__":

    # Start an infinite loop to allow continuous interaction with the system
    while True:

        # Create a new instance of the MassBooking class for this session
        # This sets up necessary attributes and loads the existing CSV file (if available)
        booking = MassBooking()
        
        # Display the introductory message and present available actions to the user
        booking.introductory_page()

        # Get the action input from the user (converted to lowercase for consistency)
        action = booking.user_introduction.lower()

        # If user wants to add a new mass intention
        if action == 'add':
            booking.adding_intention()

        # If user wants to update an existing mass intention
        elif action == 'upd':
            booking.update_intention()

        # If user wants to delete a mass intention
        elif action == 'del':
            booking.delete_intention()

        # If user wants to check booked intentions for a specific date
        elif action == 'check':
            booking.check_bookings()

        # If user chooses to exit the program
        elif action == 'exit':
            print("Thank you for using the Mass Booking Management System. Goodbye!")
            break  # Exit the infinite loop, thus ending the program

        # If the user enters an invalid option
        else:
            print("Invalid option. Please try again.")




Welcome to St. Cyprian Mass Booking Management System

Please note each entry is peculiar to a particular mass
